In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import time
import tensorflow as tf

from numpy import random
from sklearn.utils import shuffle
from tensorflow.keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2022-07-14 20:03:30.506914: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-07-14 20:03:30.545847: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-14 20:03:30.545890: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.10.2
2022-07-14 20:03:31.896106: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.10
2022-07-14 20:03:31.896193: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.10
2022-07-14 20:03:32.460145: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

# Data loader
Getting pairs of images

In [3]:
def data_loader(path):
    X = list()
    y = list()
    # all the characters in an alphabet is a single class
    y_out = 0
    # stores info about the structure of characters in X and y
    data_struct = dict()
    # going thouugh all alphabets
    for a in sorted(os.listdir(path)):
        data_struct[a] = [y_out,None]
        char_path = os.path.join(path,a)
        # going through all characters
        for char in sorted(os.listdir(char_path)):
            char_images= []
            # Going through all images in a character
            for image in sorted(os.listdir(os.path.join(char_path,char))):
                image_path = os.path.join(char_path,char,image)
                char_images.append(cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2GRAY))
                y.append(y_out)
            data_struct[a][1] = y_out
            y_out += 1
            X.append(char_images)
    X = np.array(X)
    y = np.array(y)
    return X, y, data_struct

In [4]:
train_directory = "siamese_networks/omniglot/images_background"
val_directory = "siamese_networks/omniglot/images_evaluation"
X_train,y_train, X_train_data_struct = data_loader(train_directory)
X_val,y_val, X_val_data_struct = data_loader(val_directory)

In [5]:
X_train.shape, y_train.shape

((964, 20, 105, 105), (19280,))

## Get batches of data

In [6]:
def get_batch(batch_size,data_type='train'):
    if data_type == 'train':
        X = X_train
    elif data_type == 'val':
        X = X_val
    n_data, n_samples, w, h = X.shape
    random_from_data = random.choice(n_data, size=batch_size, replace=False)
    y = np.zeros((batch_size,))
    y[batch_size//2:] = 1
    pairs = [np.zeros((batch_size,w,h,1)) for _ in range(2)]
    for i in range(batch_size):
        random_sample = random.randint(n_samples)
        pairs[0][i,:,:,:] = X[random_from_data[i],random_sample,:,:].reshape(w,h,1)
        random_from_data_2 = 0
        if i >= batch_size // 2:
            random_from_data_2 = random_from_data[i]
        else:
            random_from_data_2 = (random_from_data[i] + random.randint(1,n_data)) % n_data
        random_sample_2 = random.randint(n_samples)
        pairs[1][i,:,:,:] = X[random_from_data_2,random_sample_2,:,:].reshape(w,h,1)
    return pairs,y

# Make N shot predictions

In [7]:
def one_shot_prediction(N,data_type='val'):
    if data_type == 'train':
        X = X_train
    elif data_type == 'val':
        X = X_val
    n_data, n_samples, w, h = X.shape
    random_data = random.choice(n_data,size=(N,))
    random_sample = random.choice(n_samples,size=(N,))
    
    query = np.array([X[random_data[0],random.randint(n_samples)]]*N).reshape(N,w,h,1)
    support_set = X[random_data,random_sample].reshape(N,w,h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    
    query,support_set,targets = shuffle(query,support_set,targets)
    
    return [query,support_set], targets

In [8]:
def test_one_shot(model,N,k,data_type='val'):
    n_correct = 0
    for _ in range(k):
        X, y = one_shot_prediction(N,data_type)
        preds = model.predict(X)
        if np.argmax(y) == np.argmax(preds):
            n_correct += 1
    return round(((n_correct / k) * 100),2)

# Siamese Network

In [9]:
def siamese_model(image_shape, droupout_rate):
    left_image = Input(image_shape)
    right_image = Input(image_shape)
    
    model = Sequential()
    model.add(Conv2D(32,(5,5),input_shape=image_shape,activation='relu',kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2,strides=(2,2)))
    model.add(Dropout(droupout_rate))

    model.add(Conv2D(64,(5,5),kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2,strides=(2,2)))
    model.add(Dropout(droupout_rate))

    model.add(Conv2D(128,(5,5),kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2,strides=(2,2)))
    model.add(Dropout(droupout_rate))

    model.add(Flatten())
    
    # ----------------------------------------------------------------
    # Conversion to MLP
    # model.add(Dense(2048,activation='relu',kernel_regularizer='l2'))
    # model.add(BatchNormalization())
    # model.add(Dropout(droupout_rate))
    # model.add(Dense(1024,activation='relu',kernel_regularizer='l2'))
    # model.add(BatchNormalization())
    # model.add(Dropout(droupout_rate))
    # model.add(Dense(512,activation='relu',kernel_regularizer='l2'))
    # model.add(BatchNormalization())
    # model.add(Dropout(droupout_rate))

    model.add(Dense(256,activation='sigmoid',kernel_regularizer='l2'))

    left_embeddings = model(left_image)
    right_embeddings = model(right_image)

    L1_Layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_Dist = L1_Layer([left_embeddings,right_embeddings])
    out = Dense(1,activation='sigmoid',kernel_regularizer='l2')(L1_Dist)

    siamese_model = Model(inputs=[left_image,right_image],outputs=out)

    return siamese_model

In [10]:
droupout_rate = 0.3
initial_learning_rate = 0.05
decay_steps=4000
decay_rate=0.0001

n_data, n_samples, w, h = X_train.shape

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# initializer = tf.keras.initializers.Zeros()
model = siamese_model((w, h, 1), droupout_rate)
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)
model.summary()

2022-07-14 20:07:49.368774: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-14 20:07:49.369123: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2022-07-14 20:07:49.389146: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.10.2
2022-07-14 20:08:06.463772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-07-14 20:08:06.463813: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2022-07-14 20:08:06.463823: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2022-07-14 20:08:06.464935: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1418] Created TensorFlow device (/job:l

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 256)          2911488     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256)          0           sequential[0][0]             

# Training

In [11]:
trials = 250
N_way = 20
n_iterations = 5000
batch_size = 128

In [ ]:
losses = []
train_accuracy = []
val_accuracy = []
curr_time = time.time()
for i in range(0,n_iterations+1):
    x,y = get_batch(batch_size)
    loss = model.train_on_batch(x,y)
    if i % 100 == 0:
        losses.append(loss[0])
        train_acc_temp = test_one_shot(model,N_way,trials,'train')
        val_acc_temp = test_one_shot(model,N_way,trials,'val')
        train_accuracy.append(train_acc_temp)
        val_accuracy.append(val_acc_temp)
        print('Iteration',i,'('+str(round(time.time() - curr_time,1))+'s) - Loss:',loss[0],'Accuracy:',round(loss[1],2),'',end='')
        print('train accuracy:', train_acc_temp,'%, ',end='')
        print('val accuracy:', val_acc_temp,'%')
        curr_time = time.time()

In [ ]:
epochs_range = [i*10 for i in range(1, len(train_accuracy) + 1)]
plt.figure(figsize=(5,5))
plt.ylim(0, 90)

plt.plot(epochs_range, train_accuracy, label='Train Set')
plt.plot(epochs_range, val_accuracy, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(5,5))

plt.plot([i for i in range(len(losses))], losses, label='Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.tight_layout()
plt.show()

# Save the model

In [ ]:
version = 1
additional_info = ""
model_name = "my_model_v"+str(version)

In [ ]:
model_info = {}
model_info["version"] = 1
model_info["trials"] = str(trials)
model_info["N way"] = str(N_way)
model_info["Batch size"] = str(batch_size)
model_info["N iterations"] = str(n_iterations)
model_info["drop out"] = str(droupout_rate)
model_info["initial_learning_rate"] = str(initial_learning_rate)
model_info["decay_steps"] = str(decay_steps)
model_info["decay_rate"] = str(decay_rate)
model_info["Additional info"] = additional_info
model_info["losses"] = str(losses)
model_info["Training_accuracy"] = str(train_accuracy)
model_info["Validation_accuracy"] = str(val_accuracy)

In [ ]:
save = True
if save:
    model.save("siamese_networks/models/" + model_name)
    np.savetxt("siamese_networks/models/" + model_name+"/losses.csv", losses,delimiter =", ", fmt ='%f') 
    np.savetxt("siamese_networks/models/" + model_name+"/train_accuracy.csv", train_accuracy,delimiter =", ", fmt ='%f') 
    np.savetxt("siamese_networks/models/" + model_name+"/val_accuracy.csv", val_accuracy,delimiter =", ", fmt ='%f')

In [ ]:
import json
with open("siamese_networks/models/" + model_name + "/info.json", "w") as outfile:
    json.dump(model_info, outfile)